In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import numpy as np

In [18]:
datafile_train=r'/Users/61414/Documents/ML/Project1/P1 Data/Consumer_Complaints_train.csv'
datafile_test=r'/Users/61414/Documents/ML/Project1/P1 Data/Consumer_Complaints_test_share.csv'

cd_train=pd.read_csv(datafile_train)
cd_test=pd.read_csv(datafile_test)

In [19]:
cd_test['Consumer disputed?']=np.nan
cd_train['data']='train'
cd_test['data']='test'
cd_all=pd.concat([cd_train,cd_test],axis=0)

In [20]:
cd_all.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,data
0,2014-05-15,Credit card,NaN,Billing statement,NaN,NaN,NaN,Wells Fargo & Company,MI,48342,Older American,NaN,Web,2014-05-16,Closed with explanation,Yes,No,856103,train
1,2014-09-18,Bank account or service,(CD) Certificate of deposit,"Making/receiving payments, sending money",NaN,NaN,NaN,Santander Bank US,PA,18042,NaN,NaN,Referral,2014-09-24,Closed,Yes,No,1034666,train
2,2014-03-13,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Equifax,CA,92427,NaN,NaN,Referral,2014-04-03,Closed with non-monetary relief,Yes,No,756363,train
3,2015-07-17,Credit card,NaN,Billing statement,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,U.S. Bancorp,GA,305XX,Older American,Consent provided,Web,2015-07-17,Closed with monetary relief,Yes,No,1474177,train
4,2014-11-20,Credit card,NaN,Transaction issue,NaN,NaN,NaN,Bank of America,MA,02127,NaN,NaN,Web,2014-11-28,Closed with explanation,Yes,No,1132572,train


In [21]:
cd_all.shape

(598027, 19)

In [22]:
for col in ['Date received','Date sent to company']:
    cd_all[col]=pd.to_datetime(cd_all[col],infer_datetime_format=True)

In [23]:
cd_all['day_diff']=pd.to_numeric(cd_all['Date sent to company']-cd_all['Date received'])
cd_all.dtypes

Date received                   datetime64[ns]
Product                                 object
Sub-product                             object
Issue                                   object
Sub-issue                               object
Consumer complaint narrative            object
Company public response                 object
Company                                 object
State                                   object
ZIP code                                object
Tags                                    object
Consumer consent provided?              object
Submitted via                           object
Date sent to company            datetime64[ns]
Company response to consumer            object
Timely response?                        object
Consumer disputed?                      object
Complaint ID                             int64
data                                    object
day_diff                                 int64
dtype: object

In [24]:
for col in ['Date received','Date sent to company']:
    cd_all.drop([col],1,inplace=True)

In [25]:
for col in cd_all.select_dtypes(['object']).columns:
    print(col,':',cd_all[col].nunique())

Product : 12
Sub-product : 47
Issue : 95
Sub-issue : 68
Consumer complaint narrative : 92118
Company public response : 10
Company : 3437
State : 62
ZIP code : 27342
Tags : 3
Consumer consent provided? : 4
Submitted via : 6
Company response to consumer : 7
Timely response? : 2
Consumer disputed? : 2
data : 2


In [26]:
cd_all.isnull().sum()

Product                              0
Sub-product                     173225
Issue                                0
Sub-issue                       365685
Consumer complaint narrative    504376
Company public response         484859
Company                              0
State                             4764
ZIP code                          4774
Tags                            513950
Consumer consent provided?      428676
Submitted via                        1
Company response to consumer         0
Timely response?                     0
Consumer disputed?              119606
Complaint ID                         0
data                                 0
day_diff                             0
dtype: int64

In [27]:
for col in ['Sub-product','Sub-issue',
            'Company public response','Tags','Consumer consent provided?']:
    varname=col.replace('-','_').replace('?','').replace(" ",'_')+'_isNan'
    cd_all[varname]=np.where(pd.isnull(cd_all[col]),1,0)
    cd_all.drop([col],1,inplace=True)

In [28]:
cd_all.shape

(598027, 18)

In [29]:
for col in cd_all.select_dtypes(['object']).columns:
    print(col,':',cd_all[col].nunique())

Product : 12
Issue : 95
Consumer complaint narrative : 92118
Company : 3437
State : 62
ZIP code : 27342
Submitted via : 6
Company response to consumer : 7
Timely response? : 2
Consumer disputed? : 2
data : 2


In [30]:
for col in ['ZIP code' ]:
    cd_all.drop([col],1,inplace=True)

cd_all.shape

(598027, 17)

In [31]:
cd_all['Company'].value_counts()

Bank of America                           57887
Wells Fargo & Company                     44337
JPMorgan Chase & Co.                      36040
Equifax                                   36013
Experian                                  33622
                                          ...  
South Exchange Inc                            1
Dennis P Lee P.C., L.L.O.                     1
TFLC Trust                                    1
Quality Title, Inc.                           1
Law Offices of Goldsmith & Hull A.P.C.        1
Name: Company, Length: 3437, dtype: int64

In [32]:
for col in ['Company']:
    #print(col,"first loop")
    freqs=cd_all[col].value_counts()
    #print(col,freqs,"first loop frequency")
    k=freqs.index[freqs>1000][:-1]
    print("value of ", k)
    print(k, col,freqs,"first loop frequency k")
    for cat in k:
        name=col+'_'+cat
        cd_all[name]=(cd_all[col]==cat).astype(int)
        #print(name)
    del cd_all[col]
    print(col)

cd_all.shape

value of  Index(['Bank of America', 'Wells Fargo & Company', 'JPMorgan Chase & Co.',
       'Equifax', 'Experian', 'TransUnion Intermediate Holdings, Inc.',
       'Citibank', 'Ocwen', 'Capital One', 'Nationstar Mortgage',
       'Synchrony Financial', 'U.S. Bancorp', 'Ditech Financial LLC',
       'Navient Solutions, Inc.', 'PNC Bank N.A.', 'Encore Capital Group',
       'Amex', 'HSBC North America Holdings Inc.', 'SunTrust Banks, Inc.',
       'Discover', 'TD Bank US Holding Company',
       'Select Portfolio Servicing, Inc',
       'Portfolio Recovery Associates, Inc.', 'Citizens Financial Group, Inc.',
       'Fifth Third Financial Corporation', 'Seterus, Inc.', 'Barclays PLC',
       'ERC', 'BB&T Financial', 'M&T Bank Corporation',
       'PayPal Holdings, Inc.', 'Ally Financial Inc.', 'USAA Savings',
       'Specialized Loan Servicing LLC', 'Regions Financial Corporation',
       'AES/PHEAA', 'Santander Consumer USA Holdings Inc', 'Santander Bank US',
       'Resurgent Capital Se

(598027, 76)

In [33]:
cd_all['Consumer disputed?']=np.where(cd_all['Consumer disputed?']=="Yes",1,0)

In [34]:
k=cd_all['Issue'].value_counts()
for val in k.axes[0][0:10]:
    varname='Issue_'+val.replace(',','_').replace(' ','_')
    cd_all[varname]=np.where(cd_all['Issue']==val,1,0)
    
del cd_all['Issue']
cd_all.shape

(598027, 85)

In [35]:
for col in cd_all.select_dtypes(['object']).columns:
    print(col,':',cd_all[col].nunique())
    
cd_all.shape

Product : 12
Consumer complaint narrative : 92118
State : 62
Submitted via : 6
Company response to consumer : 7
Timely response? : 2
data : 2


(598027, 85)

In [36]:
k=cd_all['State'].value_counts()
for val in k.axes[0][0:15]:
    varname='State_'+val.replace(',','_').replace(' ','_')
    cd_all[varname]=np.where(cd_all['State']==val,1,0)
    
del cd_all['State']
cd_all.shape

(598027, 99)

In [37]:
for col in ['Product','Submitted via','Company response to consumer','Timely response?']:
    
    temp=pd.get_dummies(cd_all[col],prefix=col,drop_first=True)
    cd_all=pd.concat([temp,cd_all],1)
    cd_all.drop([col],1,inplace=True)

In [38]:
cd_all.shape

(598027, 118)

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer 

In [40]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize
lemma = WordNetLemmatizer()
my_stop=set(stopwords.words('english')+list(punctuation))

In [41]:
tfidf= TfidfVectorizer(analyzer='word',min_df=0.01,max_df=0.6,
                       stop_words=my_stop, max_features=200)

In [42]:
cd_all['Consumer complaint narrative'].fillna('missing',inplace=True)

In [43]:
tfidf.fit(cd_all['Consumer complaint narrative'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.6, max_features=200,
                min_df=0.01, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words={'!', '"', '#', '$', '%', '&', "'", '(', ')', '*',
                            '+', ',', '-', '.', '/', ':', ';', '<', '=', '>',
                            '?', '@', '[', '\\', ']', '^', '_', '`', 'a',
                            'about', ...},
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [44]:
tfidf_data=pd.DataFrame(data=tfidf.transform(cd_all['Consumer complaint narrative']).toarray(),

                       columns=tfidf.get_feature_names())

In [45]:
cd_all.reset_index(drop=True, inplace=True)
tfidf_data.reset_index(drop=True, inplace=True)

In [46]:
cd_combined = pd.concat([cd_all, tfidf_data], axis=1)

In [47]:
cd_combined.shape

(598027, 304)

In [48]:
del cd_combined['Consumer complaint narrative']

In [49]:
cd_combined.shape

(598027, 303)

In [50]:
cd_train=cd_combined[cd_combined['data']=='train']
del cd_train['data']
cd_test=cd_combined[cd_combined['data']=='test']
cd_test.drop(['Consumer disputed?','data'],axis=1,inplace=True)

In [51]:
del cd_combined

In [52]:
cd_train.shape

(478421, 302)

In [53]:
cd_test.shape

(119606, 301)

In [54]:
x=cd_train.drop(['Consumer disputed?','Complaint ID'],1)
y=cd_train['Consumer disputed?']

In [55]:
from sklearn.linear_model import LogisticRegression

In [56]:
clf=LogisticRegression()

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [49]:
submission.shape

(119606, 2)

In [ ]:
#testing the performance of model on 80:20 trai test split

In [59]:
cd_train1,cd_train2=train_test_split(cd_train,test_size=0.2,random_state=2)

In [60]:
x_train1=cd_train1.drop(['Consumer disputed?','Complaint ID'],1)
y_train1=cd_train1['Consumer disputed?']

In [61]:
x_train2=cd_train2.drop(['Consumer disputed?','Complaint ID'],1)
y_train2=cd_train2['Consumer disputed?']

In [67]:
clf.fit(x_train1,y_train1)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [68]:
test_score_logr=clf.predict_proba(x_train2)[:,1]

In [69]:
test_score_logr.shape

(95685,)

In [64]:
from sklearn.metrics import roc_auc_score

In [70]:
# score model performance on the train2 data
roc_auc_score(y_train2,test_score_logr)

0.5

In [ ]:
#cutoff with logr

In [66]:
cutoffs=np.linspace(0,1,100)
cutoffs

array([0.        , 0.01010101, 0.02020202, 0.03030303, 0.04040404,
       0.05050505, 0.06060606, 0.07070707, 0.08080808, 0.09090909,
       0.1010101 , 0.11111111, 0.12121212, 0.13131313, 0.14141414,
       0.15151515, 0.16161616, 0.17171717, 0.18181818, 0.19191919,
       0.2020202 , 0.21212121, 0.22222222, 0.23232323, 0.24242424,
       0.25252525, 0.26262626, 0.27272727, 0.28282828, 0.29292929,
       0.3030303 , 0.31313131, 0.32323232, 0.33333333, 0.34343434,
       0.35353535, 0.36363636, 0.37373737, 0.38383838, 0.39393939,
       0.4040404 , 0.41414141, 0.42424242, 0.43434343, 0.44444444,
       0.45454545, 0.46464646, 0.47474747, 0.48484848, 0.49494949,
       0.50505051, 0.51515152, 0.52525253, 0.53535354, 0.54545455,
       0.55555556, 0.56565657, 0.57575758, 0.58585859, 0.5959596 ,
       0.60606061, 0.61616162, 0.62626263, 0.63636364, 0.64646465,
       0.65656566, 0.66666667, 0.67676768, 0.68686869, 0.6969697 ,
       0.70707071, 0.71717172, 0.72727273, 0.73737374, 0.74747

In [ ]:
real = y_train1.values

In [ ]:
KS_cut=[]
for cutoff in cutoffs:
#     predicted=pd.Series([0]*len(y_train))
#     predicted[prob_score>cutoff]=1
    predicted = (prob_score>cutoff).astype(int)
    
    TP = ((real ==1) & (predicted ==1)).sum()
    TN = ((real ==0) & (predicted ==0)).sum()
    FN = ((real ==1) & (predicted ==0)).sum()
    FP = ((real ==0) & (predicted ==1)).sum()
#     df=pd.DataFrame(list(zip(y_train,predicted)),columns=["real","predicted"])
#     TP=len(df[(df["real"]==1) &(df["predicted"]==1) ])
#     FP=len(df[(df["real"]==0) &(df["predicted"]==1) ])
#     TN=len(df[(df["real"]==0) &(df["predicted"]==0) ])
#     FN=len(df[(df["real"]==1) &(df["predicted"]==0) ])

    P=TP+FN
    N=TN+FP
    KS=(TP/P)-(FP/N)
    KS_cut.append(KS)

In [ ]:
cutoff_data=pd.DataFrame(list(zip(cutoffs,KS_cut)),columns=["cutoff","KS"])

# cutoff_data

In [ ]:
KS_cutoff=cutoff_data[cutoff_data["KS"]==cutoff_data["KS"].max()]["cutoff"]
KS_cutoff

In [ ]:
# Performance on test data
prob_score_test=pd.Series(list(zip(*logr.predict_proba(x_train2)))[1])

predicted_test=pd.Series([0]*len(y_train2))
predicted_test[prob_score_test>float(KS_cutoff)]=1

df_test=pd.DataFrame(list(zip(y_train2,predicted_test)),columns=["real","predicted"])

k=pd.crosstab(df_test['real'],df_test["predicted"])
print('confusion matrix :\n \n ',k)
TN=k.iloc[0,0]
TP=k.iloc[1,1]
FP=k.iloc[0,1]
FN=k.iloc[1,0]
P=TP+FN
N=TN+FP

In [ ]:
=========================================================================================

In [ ]:
###grid search

In [71]:
from sklearn.model_selection import GridSearchCV

In [72]:
params={'class_weight':['balanced',None],
        'penalty':['l1','l2'],
        'C':np.linspace(0.01,1000,10)}

In [73]:
grid_search=GridSearchCV(clf,param_grid=params,cv=5,scoring="roc_auc")

In [106]:
grid_search.fit(x,y)

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([1.0000e-02, 1.1112e+02, 2.2223e+02, 3.3334e+02, 4.4445e+02,
       5.5556e+02, 6.6667e+02, 7.7778e+02, 8.8889e+02, 1.0000e+03]),
                         'class_weight': ['balanced', None],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False

In [107]:
grid_search.best_estimator_

LogisticRegression(C=0.01, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [108]:
logr_gs=grid_search.best_estimator_

In [109]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.10f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [110]:
report(grid_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.5447201040 (std: 0.00175)
Parameters: {'C': 0.01, 'class_weight': 'balanced', 'penalty': 'l2'}

Model with rank: 1
Mean validation score: 0.5447201040 (std: 0.00175)
Parameters: {'C': 111.12, 'class_weight': 'balanced', 'penalty': 'l2'}

Model with rank: 1
Mean validation score: 0.5447201040 (std: 0.00175)
Parameters: {'C': 222.23, 'class_weight': 'balanced', 'penalty': 'l2'}

Model with rank: 1
Mean validation score: 0.5447201040 (std: 0.00175)
Parameters: {'C': 333.34, 'class_weight': 'balanced', 'penalty': 'l2'}

Model with rank: 1
Mean validation score: 0.5447201040 (std: 0.00175)
Parameters: {'C': 444.45, 'class_weight': 'balanced', 'penalty': 'l2'}

Model with rank: 1
Mean validation score: 0.5447201040 (std: 0.00175)
Parameters: {'C': 555.56, 'class_weight': 'balanced', 'penalty': 'l2'}

Model with rank: 1
Mean validation score: 0.5447201040 (std: 0.00175)
Parameters: {'C': 666.67, 'class_weight': 'balanced', 'penalty': 'l2'}

Model wi

In [111]:
test_score_gs=logr_gs.predict_proba(x_train2)[:,1]

In [112]:
roc_auc_score(y_train2,test_score_gs)

0.5287059355830243

In [ ]:
=============================================================================================

In [ ]:
#KFold

In [113]:
from sklearn.model_selection import KFold

In [114]:
alphas=[0.0001,0.0002,0.0005,0.001,0.005,0.01,0.05, 0.1,0.5, 1, 10, 100,500,1000]

auc_10cv =[]
for a in alphas:
    logr=LogisticRegression(C=a,penalty="l2",class_weight="balanced",random_state=2, max_iter=100)
    kf = KFold( n_splits=10)
    auc_for_one_alpha = []
    for train_idx, test_idx in kf.split(x):
        kfold_train_x = x.loc[train_idx]
        kfold_train_y = y.loc[train_idx]
        
        kfold_test_x = x.loc[test_idx]
        kfold_test_y = y.loc[test_idx]

        logr.fit(kfold_train_x, kfold_train_y)

        auc_score_held_out_fold = roc_auc_score(kfold_test_y, logr.predict(kfold_test_x))
        auc_for_one_alpha.append(auc_score_held_out_fold)

    auc_10cv.append(np.mean(auc_for_one_alpha))
best_alpha = alphas[np.argmax(np.array(auc_10cv))]
best_alpha

0.0001

In [115]:
logr_KFold=LogisticRegression(C=best_alpha,penalty="l2",class_weight="balanced",random_state=2)

In [116]:
logr_KFold.fit(x_train1,y_train1)

LogisticRegression(C=0.0001, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=2, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [117]:
test_score_KFold=logr_KFold.predict_proba(x_train2)[:,1]

In [118]:
roc_auc_score(y_train2,test_score_KFold)

0.5287059355830243

In [ ]:
===========================================================================================

In [ ]:
#Decision tree

In [119]:
from sklearn import tree
from sklearn.model_selection import RandomizedSearchCV
%matplotlib inline

In [120]:
from sklearn.tree import DecisionTreeClassifier

In [121]:
params={ 'class_weight':[None,'balanced'], 
        'criterion':['entropy','gini'],
        'max_depth':[None,5,10,15,20,30,50,70],
            'min_samples_leaf':[1,2,5,10,15,20], 
            'min_samples_split':[2,5,10,15,20]
       }

In [122]:
clf_dt=DecisionTreeClassifier()

In [123]:
random_search=RandomizedSearchCV(clf_dt,cv=10,
                                 param_distributions=params,
                                 scoring='roc_auc',
                                 n_iter=10
                                    )

In [124]:
random_search.fit(x,y)

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=None,
         

In [125]:
random_search.cv_results_

{'mean_fit_time': array([43.54669998,  5.62441862,  6.23923187, 34.35096023, 11.42767706,
        18.76213877, 15.4892694 , 20.51095994, 17.08413546,  9.74099782]),
 'std_fit_time': array([2.62210272, 0.22009256, 0.15602784, 2.02989751, 0.55133505,
        0.45993825, 0.21570623, 1.92157247, 0.10662923, 0.10651358]),
 'mean_score_time': array([0.1037801 , 0.07923324, 0.07610636, 0.09794774, 0.08942225,
        0.08194597, 0.12331834, 0.084255  , 0.08190417, 0.07644684]),
 'std_score_time': array([0.01163691, 0.01130412, 0.00536499, 0.01138288, 0.02313099,
        0.0050015 , 0.1183767 , 0.0070482 , 0.00523953, 0.00549918]),
 'param_min_samples_split': masked_array(data=[10, 20, 15, 20, 15, 5, 5, 2, 20, 15],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_leaf': masked_array(data=[2, 5, 1, 1, 5, 20, 20, 2, 2, 5],
              mask=[False, False, False, 

In [126]:
random_search.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=10, max_features=None,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=5,
                       min_samples_split=15, min_weight_fraction_leaf=0.0,
                       presort='deprecated', random_state=None,
                       splitter='best')

In [127]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.6173933438 (std: 0.00367)
Parameters: {'min_samples_split': 15, 'min_samples_leaf': 5, 'max_depth': 10, 'criterion': 'entropy', 'class_weight': 'balanced'}

Model with rank: 2
Mean validation score: 0.6172414545 (std: 0.00353)
Parameters: {'min_samples_split': 15, 'min_samples_leaf': 5, 'max_depth': 10, 'criterion': 'gini', 'class_weight': 'balanced'}

Model with rank: 3
Mean validation score: 0.6156714996 (std: 0.00398)
Parameters: {'min_samples_split': 5, 'min_samples_leaf': 20, 'max_depth': 15, 'criterion': 'entropy', 'class_weight': 'balanced'}

Model with rank: 4
Mean validation score: 0.6099797819 (std: 0.00278)
Parameters: {'min_samples_split': 5, 'min_samples_leaf': 20, 'max_depth': 20, 'criterion': 'entropy', 'class_weight': None}

Model with rank: 5
Mean validation score: 0.6061996252 (std: 0.00269)
Parameters: {'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': 20, 'criterion': 'entropy', 'class_weight': 'balanced'}



In [128]:
dtree=random_search.best_estimator_

In [129]:
dtree.fit(x,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=10, max_features=None,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=5,
                       min_samples_split=15, min_weight_fraction_leaf=0.0,
                       presort='deprecated', random_state=None,
                       splitter='best')

In [130]:
test_score_dt=dtree.predict_proba(x_train2)[:,1]

In [131]:
roc_auc_score(y_train2,test_score_dt)

0.6293365527699974

In [135]:
#clf_dt.fit(x,y)

In [136]:
prediction_dtree=np.where(dtree.predict(cd_test.drop(['Complaint ID'],1))==1,"Yes","No")

In [137]:
submission_dt=pd.DataFrame(list(zip(cd_test['Complaint ID'],list(prediction_dtree))),
                       columns=['Complaint ID','Consumer disputed?'])

In [89]:
import os
os.getcwd()

'C:\\Users\\61414\\Documents\\ML\\Project1'

In [102]:
submission_dt

,Complaint ID,Consumer disputed?
0,675956,No
1,1858795,No
2,32637,No
3,1731374,No
4,501487,No
...,...,...
119601,153482,No
119602,2023523,No
119603,1413678,No
119604,732458,No


In [134]:
submission_dt.to_csv('sample_submission_run4.csv',index=False)

In [104]:
test_score_dt=dtree.predict_proba(x)[:,1]

In [105]:
roc_auc_score(y,test_score_dt)

0.6285781165701678

In [ ]:
==============================================================================